In [1]:
from sklearn.cluster import KMeans
import numpy as np
import datetime
import pandas as pd
import random

In [2]:
X=pd.read_csv('nifty.csv')
len(X)

2966

In [3]:
def data1():
    X=pd.read_csv('nifty.csv')
    X['c-o']=(X.Close-X.Open)/X.Open*100
    X['candle']=np.where(X['c-o'].values<-1,0,np.where(X['c-o'].values<-0.5,1,np.where(X['c-o'].values<0,2,np.where(X['c-o'].values<0.5,3,np.where(X['c-o'].values<1,4,5)))))
    X = X.dropna(axis = 0, how ='any')
    X = X.reset_index()
    X['cashopen']=0
    X['cash']=0
    X['action']=""
    X['diff']=""
    X['quantity']=""
    return X
    

In [4]:
X=data1()
print(len(X))

2966


In [5]:
len(X)

2966

In [6]:
X['candle'].value_counts()

candle
2    848
3    812
1    434
4    362
0    298
5    212
Name: count, dtype: int64

In [7]:
X.head()

,index,Unnamed: 0,date,Close,High,Low,Open,Volume,weekday,month,c-o,candle,cashopen,cash,action,diff,quantity
0,0,0,21-01-2013,6082.299805,6094.350098,6065.100098,6085.750000,130900,0,1,-0.056693,2,0,0,,,
1,1,1,22-01-2013,6048.500000,6101.299805,6040.500000,6080.149902,129000,1,1,-0.520545,1,0,0,,,
2,2,2,23-01-2013,6054.299805,6069.799805,6021.149902,6052.850098,137000,2,1,0.023951,3,0,0,,,
3,3,3,24-01-2013,6019.350098,6065.299805,6007.850098,6046.200195,185200,3,1,-0.444082,2,0,0,,,
4,4,4,25-01-2013,6074.649902,6080.549805,6014.450195,6024.500000,147600,4,1,0.832433,4,0,0,,,


In [7]:
def getState(t):    
    return(X['candle'][t])  

In [8]:
class Agent:
    def __init__(self):
           
        self.gamma = np.load('rl/gamma100.npy')
        self.Q = np.load('rl/Qtable100.npy')
        self.learning_rate=np.load('rl/learning_rate100.npy')
        self.epsilon = np.load('rl/epsilon100.npy')
        print(self.epsilon)
    def getAction(self,state):
        return np.argmax(self.Q[state,])  
    
    # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    def update(self,state,action,reward,next_state,next_action):
            self.Q[state,action]= self.Q[state,action] + self.learning_rate * (reward + self.gamma * np.max(self.Q[next_state, :])
                                                                - self.Q[state,action])

        

In [9]:
episode_count=1
agent=Agent()
agent.Q
m1=0
m2=0
for e in range(1,episode_count+1):
    X=data1()
    
    data_length_start=2436
    data_length=2466
    state = getState(data_length_start-1)
    action = agent.getAction(state)
    
  
    capital=X.loc[data_length_start,'Open']*10
    margin=X.loc[data_length_start,'Open']*10
    X.loc[2,'cashopen']=margin 
    brok_rate=0.0000
    #brok_rate=0.0
    
    print("Episod {}/{}".format(e,episode_count))
    count=0
    for t in range(data_length_start,data_length-1):
                
        reward=0                        
         
        if action == 1:
            X.loc[t,'cashopen']=margin 
            quantity=margin//X.loc[t,'Open']
            X.loc[t,'quantity']=quantity
            margin=margin%X.loc[t,'Open']
            tbrokerage=(X.loc[t,'Open']+X.loc[t,'Close'])*brok_rate
            tprofit=(X.loc[t,'Close']-X.loc[t,'Open']-tbrokerage)*quantity
            margin=margin+tprofit+X.loc[t,'Open']*quantity
            X.loc[t,'cash']=margin         
            
            #print('Long: open price @',X.loc[t,'Open'],'Close price @',X.loc[t,'Close'], 'profit=',tprofit)
            X.loc[t,'action']="long"
            X.loc[t,'diff']=(X.loc[t,'Close']-X.loc[t,'Open']-tbrokerage)*quantity
            count=count+1
            reward= max((np.log(X.loc[t,'Close'])-np.log(X.loc[t,'Open']+tbrokerage)),0)
       
        elif action == 2:
            X.loc[t,'cashopen']=margin 
            quantity=margin//X.loc[t,'Open']
            X.loc[t,'quantity']=(-quantity)
            margin=margin%X.loc[t,'Open']
            tbrokerage=(X.loc[t,'Open']+X.loc[t,'Close'])*brok_rate
            tprofit=(X.loc[t,'Open']-X.loc[t,'Close']-tbrokerage)*quantity
            #print('in Short')
            margin=margin+tprofit+X.loc[t,'Open']*quantity
            X.loc[t,'cash']=margin         
            last_action='Short'
            #print('Short: Close price @',X.loc[t,'Close'],'Open price @',X.loc[t,'Open'], 'profit=',tprofit)
            X.loc[t,'action']="short"
            X.loc[t,'diff']=(X.loc[t,'Open']-X.loc[t,'Close']-tbrokerage)*quantity
            count=count+1
            reward= max((np.log(X.loc[t,'Open'])-np.log(X.loc[t,'Close']+tbrokerage)),0)
        else:
            X.loc[t,'cash']=margin
            X.loc[t,'action']="Hold"
            X.loc[t,'diff']=0
            reward=0                                  
              
        next_state=getState(t)
        next_action=agent.getAction(next_state)
        agent.update(state,action,reward,next_state,next_action)
        state=next_state
        action=next_action
        
        if t==data_length-2:
                print("\n***********************\n")
                print("Profit is ",(margin-capital))
                print("Initial Capital is ",capital)    
                print("Current Capital is ",margin)
                roi=((margin-capital)/capital)*100
                print("%ROI is ",roi)
                bnh=((X.loc[t,'Close']-X.loc[data_length_start,'Open'])/X.loc[data_length_start,'Open'])*100
                print("% Buy and Hold=",bnh)
                print("count",count)
                print("qtable",agent.Q)
                print("epsilon",agent.epsilon)
                print("--------------------------------")
                       
       
    
    m3=roi
    if m3>m2:
        m1=e
        m2=m3
#To save model  
if e%10==0:
    np.save('rl/test/Qtable'+ str(e)+str('.npy'), agent.Q)
    np.save('rl/test/epsilon'+ str(e)+str('.npy'), agent.epsilon)
    np.save('rl/test/gamma'+ str(e)+str('.npy'), agent.gamma)
    np.save('rl/test/learning_rate'+ str(e)+str('.npy'), agent.learning_rate)
    X.to_csv('csv1/AfterTest'+str(e)+str('.csv'))
print("e={},roi={}".format(m1,m2))

0.00998645168764533
Episod 1/1

***********************

Profit is  12936.613420000067
Initial Capital is  179776.50389999998
Current Capital is  192713.11732000005
%ROI is  7.195942261284606
% Buy and Hold= -2.0428142278497163
count 29
qtable [[0.00585441 0.00809641 0.01015196]
 [0.00573818 0.00989568 0.00716493]
 [0.00558976 0.00691121 0.00947369]
 [0.00583713 0.00678079 0.0092166 ]
 [0.00557333 0.00716788 0.00884673]
 [0.00535027 0.00782478 0.01083407]]
epsilon 0.00998645168764533
--------------------------------
e=1,roi=7.195942261284606


C:\Users\IIITNAGPUR\AppData\Local\Temp\ipykernel_19404\1380074554.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '179776.50389999998' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[2,'cashopen']=margin
C:\Users\IIITNAGPUR\AppData\Local\Temp\ipykernel_19404\1380074554.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '178238.85741' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[t,'cash']=margin


In [ ]:
agent.Q

In [ ]:
%matplotlib inline
#%pylab inline
fig = plt.figure()
ax=X.loc[data_length_start:data_length-2,'cash'].plot(grid="True",figsize=(10,5), title='Nifty')
ax.set_xlabel("Trading Days")
ax.set_ylabel("cash")
plt.show()
#fig.savefig('baroda_close.png')